In [3]:
import requests
import json
import itertools


In [4]:
def readJson(file):
    with open(file,"r") as f:
        rf=json.load(f)
    return rf

In [5]:
# Initial requests from API url (23.08.19)
config = readJson("config.json")
client_id=config["client_id"]
elev_key = config["elevation_key"]
test_area = config["test_area"]

#Mapillary
def requestImageById(image_key):
    request = requests.get("https://a.mapillary.com/v3/images/"+image_key+"?client_id="+client_id)
    return request.json()

def requestSequencesByBbox(ul, lr):
    # i prefer [lat, lng], so have to keep that in mind
    # request of bbox is box=minx,miny,maxx,maxy=ul_lng, lr_lat, lr_lng, ul_lat
    request = requests.get(f"https://a.mapillary.com/v3/sequences?bbox={ul[1]},{lr[0]},{lr[1]},{ul[0]}&per_page=10000&client_id={client_id}")
    return request.json()

def requestSequenceById(sequence_key):
    request = requests.get("https://a.mapillary.com/v3/sequences/"+sequence_key+"?client_id="+client_id)
    return request.json()

def requestImagesBySequence(sequence_key):
    request = requests.get("https://a.mapillary.com/v3/images/?sequence_keys="+sequence_key+"&per_page=10000&client_id="+client_id)
    return request.json()

In [38]:
# Mapillary API allows max 1000 sequences per area, so if the number of sequences>1000, i split in 4 and request for new areas
def get_all_sequences_in_area(bbox):
    ul, lr = bbox
    areas = [[ul, lr]]
    done = []
    max_retrieved = 1000
    
    while(True):
        new_areas = []
        for (ul_i, lr_i) in areas:   
            request = requestSequencesByBbox(ul_i, lr_i)
            if(len(request["features"])<max_retrieved):
                done.append(request["features"])
            else:
                split_lat = ul_i[0]+(lr_i[0]-ul_i[0])/2
                split_lng = lr_i[1]+(ul_i[1]-lr_i[1])/2
                split = [split_lat, split_lng]
                new_areas.append([ul_i,split])
                new_areas.append([[split[0], ul_i[1]],[lr_i[0], split[1]]])
                new_areas.append([[ul_i[0], split[1]],[split[0], lr_i[1]]])
                new_areas.append([split, lr_i])
        if(len(new_areas)==0):
            break
        else:
            areas = new_areas
    
    def remove_dupes(l):
        seen = set()
        new_l = []
        for d in l:
            if d["properties"]["key"] not in seen:
                seen.add(d["properties"]["key"])
                new_l.append(d)
        return new_l
    
    return {
        "type":"FeatureCollection",
        "features":remove_dupes(list(itertools.chain.from_iterable(done)))
    }

def write_data(data):
    to_file = "mapillary_data.json"
    features = []
    for i, seq in enumerate(data["features"]):
        _id =seq["properties"]["key"]
        images = requestImagesBySequence(_id)["features"]
        features.append({
            "id":_id,
            "images":images
        })
        
    with open(to_file, "w+") as f:
        json.dump({
            "type":"FeatureCollection",
            "features":features
        }, f)

In [35]:
data = get_all_sequences_in_area(test_area)

In [43]:
data["features"][0]

{'type': 'Feature',
 'properties': {'camera_make': 'samsung',
  'captured_at': '2019-11-09T12:57:40.440Z',
  'coordinateProperties': {'cas': [100.95,
    103.123,
    101.874,
    101.874,
    100.877,
    100.022,
    100.841,
    101.106,
    100.904,
    100.904,
    99.5515,
    99.5515,
    100.119,
    102.448,
    102.448,
    103.691,
    103.236,
    102.343,
    102.343,
    107.282,
    107.282,
    107.211,
    110.405,
    111.02,
    111.02,
    108.915,
    108.915,
    111.718,
    113.415,
    113.768,
    116.711,
    116.528,
    115.599,
    115.599,
    120.459,
    120.459,
    121.425,
    121.425,
    120.841,
    121.676,
    125.341,
    125.341,
    128.448,
    128.171,
    125.892,
    125.081,
    126.486,
    129.43,
    131.424,
    131.423,
    125.688,
    125.688,
    125.443,
    128.601,
    128.601,
    126.602,
    126.602,
    123.919,
    124.675,
    125.761,
    126.041,
    126.041,
    124.811,
    124.483,
    125.465,
    123.78,
    125.1

In [44]:
write_data(data)